In [1]:
import numpy as np
from os.path import isfile,join
from os import listdir
from scipy import signal as sg
from scipy.signal import butter,lfilter,freqz
import matplotlib.pyplot as plt
from scipy import hamming
import soundfile as sf
import scipy
from python_speech_features import mfcc
from python_speech_features import logfbank
import librosa
from scipy.fftpack import fft
from scipy.io import wavfile
import soundfile as sf
from pydub import AudioSegment
from pydub.utils import make_chunks
import math
import tensorflow
from keras.utils import np_utils
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.models import model_from_json
from keras.layers import Dense

from keras import backend
from keras.layers import Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import keras
from keras import backend as K
from keras.models import Sequential,model_from_json
from keras.layers import Conv2D,Conv1D,MaxPooling1D,GlobalAveragePooling1D,GlobalMaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten,Dropout
from keras import optimizers, callbacks
import numpy as np
from keras.layers import Dense,Activation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,img_to_array
#from sklearn.preprocessing import StandardbScaler

c:\users\alimi\appdata\local\programs\python\python36\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


In [2]:
babycry=[f for f in listdir('/Users/User/100baby_cry/370CRY') if isfile(join('/Users/User/100baby_cry/370CRY', f))]

babysilence=[f for f in listdir('/Users/User/baby/baby_cry/901 - Silence') if isfile(join('/Users/User/baby/baby_cry/901 - Silence', f))]
babynoise=[f for f in listdir('/Users/User/baby/baby_cry/902 - Noise') if isfile(join('/Users/User/baby/baby_cry/902 - Noise', f))]
babylaugh=[f for f in listdir('/Users/User/baby/baby_cry/903 - Baby laugh') if isfile(join('/Users/User/baby/baby_cry/903 - Baby laugh', f))]
babynew=[f for f in listdir('/Users/User/baby/audio') if isfile(join('/Users/User/baby/audio', f))]


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/User/100baby_cry/370CRY'

In [ ]:
baby_cry=[]
for each in range(0,len(babycry)) :
    cry='/Users/User/100baby_cry/370CRY' + '/' + babycry[each] 
    baby_cry.append(cry)


In [ ]:
baby_silence=[]
for each in range(0,len(babysilence)) :
    silence='/Users/User/baby/baby_cry/901 - Silence' + '/' + babysilence[each] 
    baby_silence.append(silence)


In [ ]:
baby_noise=[]
for each in range(0,len(babynoise)) :
    noise='/Users/User/baby/baby_cry/902 - Noise' + '/' + babynoise[each] 
    baby_silence.append(noise)


In [ ]:
baby_laugh=[]
for each in range(0,len(babylaugh)) :
    laugh='/Users/User/baby/baby_cry/903 - Baby laugh' + '/' + babylaugh[each] 
    baby_laugh.append(laugh)
  

In [ ]:
baby_new=[]
for each in range(0,len(babynew)) :
    unknown='/Users/User/baby/audio' + '/' + babynew[each] 
    baby_new.append(unknown)

In [ ]:
nonbabycry = baby_silence + baby_noise + baby_laugh + baby_new

In [ ]:
len(nonbabycry)

In [ ]:
len(baby_cry)

In [ ]:
x=[1,4,6,8,9,10,2,4,5]
#x=np.array(x)
np.array_split(x,2)

In [ ]:
def butter_lowpass(cutoff,fs,order=5):
    nyq=0.5*fs
    normal_cutoff=cutoff/nyq
    b,a=butter(order,normal_cutoff,btype='low',analog=False)
    return b,a

In [ ]:
def butter_lowpass_filter(data,cutoff,fs,order=5):
    b,a=butter_lowpass(cutoff,fs,order=order)
    y=lfilter(b,a,data)
    return y

In [ ]:
def feature(soundfile):
    s,r=sf.read(soundfile)
    s=butter_lowpass_filter(s,11025,44100,order=3)
    x=np.array_split(s,64)
    print(x)
    
    logg=[]
    for i in x:
             
             xx=np.mean(logfbank(i,r,nfilt=40,nfft=1103),axis=0)
             logg.append(xx)
        
    return  logg  

In [ ]:
r=feature(baby_cry[10])

In [ ]:
len(r)

In [ ]:
#j=np.mean(r,axis=0)
np.shape(r)

In [ ]:
#np.savetxt("deep.csv",r,delimiter=",")

In [ ]:
data=np.zeros((669,64,40))
output=[]
for i in range(0,len(baby_cry)):
       #print(baby_cry[i])
       r=feature(baby_cry[i])
       
       #r=list(r)
       data[i,:,:]=r
       output.append(1) 
        
all=baby_cry+nonbabycry
for i in range(len(baby_cry)-1,len(all)-1):
       #r=list(r)
      # print(all[i]) 
       r=feature(all[i])
       
       data[i,:,:]=r
       output.append(0)  

In [ ]:
a = baby_cry[0]

In [ ]:
data.shape

In [ ]:
len(data)

In [ ]:
len(output)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, output, test_size=0.30, random_state=None)

In [ ]:
from keras.layers import Reshape, Flatten, Dropout, Concatenate, BatchNormalization

model = Sequential()

model.add(Conv1D(64, 2, activation='relu', input_shape=(64, 40)))
model.add(Conv1D(40,2,padding='valid', activation='relu',strides=1))

model.add(Dropout(0.2))


model.add(GlobalMaxPooling1D())
model.add(Dense(40))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=0.001, mode=0, momentum=0.9, weights=None))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, nb_epoch=1000, validation_split=0.3,batch_size=10, shuffle=False,verbose=2)
#model.fit(data,output, epochs=10000, batch_size=10, verbose=1)

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pred.round()))
print('\n')
print(classification_report(y_test, y_pred.round()))

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
with open('ncnn1.json','w') as f:
    f.write(model.to_json())

In [ ]:
model.save_weights("ncnn1.h5")

In [ ]:
feat=feature('201104291843.wav')
d=np.zeros((32,40))
for i in range(len(feat)):
    d[i,:]=feat[i]
X=np.expand_dims(d,axis=0)    
model.predict_classes(X)[0][0]

In [ ]:
np.shape(X)